<a href="https://colab.research.google.com/github/kanavgoyal781/Gen_Ai/blob/Step-1-Choose_a_Model/Fine_Tuning_LLMs_with_QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!nvidia-smi

Fri Aug  1 19:34:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             48W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
# Install core dependencies for LoRA + supervised fine-tuning (SFTTrainer setup)
%pip install -q --upgrade \
  transformers \
  datasets \
  peft==0.8.2 \
  accelerate \
  bitsandbytes \
  trl==0.8.6 \
  wandb \
  evaluate \
  rouge_score


Step 1: Choose a Model (Compatible with LoRA Adapters)

In [5]:
from huggingface_hub import login
login()  # Then paste your updated access token